In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from zipfile import ZipFile
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from pathlib import Path
import operator

f = open('wudi.json',)
wudi = json.load(f)
wudi = {int(k):v for k, v in wudi.items()}


f = open('mudi.json',)
mudi = json.load(f)
mudi = {int(k):v for k, v in mudi.items()}
f.close()

num_users = len(np.unique(np.array(list(wudi.keys()))))
num_movies = len(np.unique(np.array(list(mudi.keys()))))

In [2]:


EMBEDDING_SIZE = 50

class RecommenderNet(keras.Model):
    def __init__(self, num_users, num_movies, embedding_size, **kwargs):
        super(RecommenderNet, self).__init__(**kwargs)
        self.num_users = num_users
        self.num_movies = num_movies
        self.embedding_size = embedding_size
        self.user_embedding = layers.Embedding(
            num_users,
            embedding_size,
            embeddings_initializer="he_normal",
            embeddings_regularizer=keras.regularizers.l2(1e-6),
        )
        self.user_bias = layers.Embedding(num_users, 1)
        self.movie_embedding = layers.Embedding(
            num_movies,
            embedding_size,
            embeddings_initializer="he_normal",
            embeddings_regularizer=keras.regularizers.l2(1e-6),
        )
        self.movie_bias = layers.Embedding(num_movies, 1)

    def call(self, inputs):
        user_vector = self.user_embedding(inputs[:, 0])
        user_bias = self.user_bias(inputs[:, 0])
        movie_vector = self.movie_embedding(inputs[:, 1])
        movie_bias = self.movie_bias(inputs[:, 1])
        dot_user_movie = tf.tensordot(user_vector, movie_vector, 2)
        # Add all the components (including bias)
        x = dot_user_movie + user_bias + movie_bias
        # The sigmoid activation forces the rating to between 0 and 1
        return tf.nn.sigmoid(x)

In [3]:
M = RecommenderNet(num_users, num_movies, 50)
M.load_weights("material_recommed")


In [4]:
##### Input #########

one_user_id=29606863  ### existing_wholeasler_id

#################

one_user_wholesaler=[]
for i in range(num_movies):
    one_user_wholesaler.append(one_user_id)
one_user_material=[]
for i in range(num_movies):
    one_user_material.append(i)

one_user=pd.DataFrame({'wholesaler':one_user_wholesaler})
one_user['material']=one_user_material
one_user["wholesaler"] = [wudi[x] for x in one_user["wholesaler"]]

key_list_1 = list(mudi.keys())
val_list_1 = list(mudi.values())

preds=[]
for i in range(len(one_user)):
    preds.append((float(M.predict(np.asarray([one_user.iloc[i]]))[0]), key_list_1[val_list_1.index(one_user['material'][i])])) 

preds
# preds=dict(sorted(preds.items(), key=operator.itemgetter(1), reverse=True)[:10])
# ratings_for_all = pd.DataFrame(preds)
# ratings_for_all

[(0.7261511087417603, 10946),
 (0.6452205777168274, 19898),
 (0.24419325590133667, 80176),
 (0.5618172287940979, 66989),
 (0.3913574814796448, 3337),
 (0.26433271169662476, 48859),
 (0.2863859534263611, 29777),
 (0.39325153827667236, 33335),
 (0.5065833926200867, 5983),
 (0.7008898854255676, 3372),
 (0.7717363834381104, 11574),
 (0.5860069990158081, 20614),
 (0.4305846095085144, 59874),
 (0.5768035054206848, 71317),
 (0.28531673550605774, 63311),
 (0.6654099225997925, 9974),
 (0.49095234274864197, 48537),
 (0.24306681752204895, 37961),
 (0.4408200979232788, 59847),
 (0.21474620699882507, 3338),
 (0.5253368020057678, 10947),
 (0.574744462966919, 6013),
 (0.596153736114502, 59848),
 (0.32221537828445435, 77313),
 (0.263028085231781, 76623),
 (0.6204102635383606, 10965),
 (0.2792638838291168, 66839),
 (0.3438469171524048, 10957),
 (0.3170102834701538, 59873),
 (0.6506699919700623, 3410),
 (0.321831613779068, 55357),
 (0.4556655287742615, 16550),
 (0.3333038091659546, 16519),
 (0.334212124